# Fine-tuning a Model for Masked Language Modeling (MLM) Exam

In this exam, you will be tasked with performing dataset preprocessing and fine-tuning a model for a masked language modeling task. Complete each step carefully according to the instructions provided.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `bert-base-uncased` for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/math_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

# Import Libraries

In [1]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling


nltk.download('stopwords')
nltk.download('english')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Error loading english: Package 'english' not found in
[nltk_data]     index
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [3]:
ds = load_dataset("CUTD/math_df")
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


math_df.csv:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text'],
        num_rows: 10000
    })
})

In [4]:
ds = ds.remove_columns('Unnamed: 0')
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 10000
    })
})

In [5]:
ds = ds['train']
ds

Dataset({
    features: ['text'],
    num_rows: 10000
})

# Step 2: Load the Pretrained Model and Tokenizer

Use a pre-trained model and tokenizer for this task. Initialize both in this step.

In [6]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Step 3: Preprocess the Dataset

Define a preprocessing function that tokenizes the text data and prepares the inputs for the model. Ensure that you truncate the sequences to a maximum length of 512 tokens and pad them appropriately.

**Bonus**: If you performed more comprehensive preprocessing, such as removing links, converting text to lowercase, or applying additional preprocessing techniques.

In [7]:
# print an instance of the data to take a look
ds[0]

{'text': "A software engineer who disagrees with the established computer scientist's methodologies and approaches"}

### **Text Cleaning**

In [8]:
stop_words = set(stopwords.words('english'))
punctuations = '''`÷×؛<>_()*&^%][.ـ،/:"!?..,'{}~¦+|!”…“–ـ'''
stemmer = PorterStemmer()

def text_cleaning(text):

  # Removing links (URLs)
  text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

  # Removing special characters and punctuations
  text = re.sub(r'[^\w\s]', '', text)
  pun = str.maketrans('', '', punctuations)
  text = text.translate(pun)

  # Lowercasing
  text = text.lower()

  # Removing English stopwords
  word_tokens = word_tokenize(text)
  filtered_text = [word for word in word_tokens if word not in stop_words]

  # Stemming
  stemmed_words = []
  for word in word_tokens:
    stemmed_words.append(stemmer.stem(word))
  stemmed_words = ' '.join(filtered_text)
  return stemmed_words

cleaned_text = ds.map(lambda x: {'text': text_cleaning(x['text'])})

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [9]:
# Befor preprocessing
ds[1]

{'text': 'A German literature college student who is interested in historical contexts of works and the personalities of authors, but normally dislikes theatrical works.'}

In [10]:
# After preprocessing
cleaned_text[1]

{'text': 'german literature college student interested historical contexts works personalities authors normally dislikes theatrical works'}

### **Tokenization**

In [11]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]] , max_length=512, padding="max_length", truncation=True)

tokenized_data = cleaned_text.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=cleaned_text.column_names)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [12]:
tokenized_data[0]

{'input_ids': [101,
  1055,
  1051,
  1042,
  1056,
  1059,
  1037,
  1054,
  1041,
  1041,
  1050,
  1043,
  1045,
  1050,
  1041,
  1041,
  1054,
  1040,
  1045,
  1055,
  1037,
  1043,
  1054,
  1041,
  1041,
  1055,
  1041,
  1055,
  1056,
  1037,
  1038,
  1048,
  1045,
  1055,
  1044,
  1041,
  1040,
  1039,
  1051,
  1049,
  1052,
  1057,
  1056,
  1041,
  1054,
  1055,
  1039,
  1045,
  1041,
  1050,
  1056,
  1045,
  1055,
  1056,
  1055,
  1049,
  1041,
  1056,
  1044,
  1051,
  1040,
  1051,
  1048,
  1051,
  1043,
  1045,
  1041,
  1055,
  1037,
  1052,
  1052,
  1054,
  1051,
  1037,
  1039,
  1044,
  1041,
  1055,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

# Step 4: Data Splitting

In [13]:
dataset = tokenized_data.train_test_split(test_size= 0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [14]:
train_data = dataset['train']
validation_data = dataset['test']

In [15]:
train_data

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8000
})

In [16]:
validation_data

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2000
})

# Step 5: Define Training Arguments

Set up the training configuration, including parameters like learning rate, batch size, number of epochs, and weight decay.

In [17]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="pt")

In [18]:
training_args = TrainingArguments(
    output_dir="./our_model",
    eval_strategy="epoch",
    learning_rate=5e-5,
    num_train_epochs=1,
    weight_decay=0.01,
)

# Step 6: Initialize the Trainer

Initialize the Trainer using the model, training arguments, and datasets (both training and evaluation).

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_data,
    eval_dataset = validation_data,
    data_collator=data_collator,
)

# Step 7: Fine-tune the Model

Run the training process using the initialized Trainer to fine-tune the model on the masked language modeling task.

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.529900,1.373551


TrainOutput(global_step=1000, training_loss=1.726384521484375, metrics={'train_runtime': 985.082, 'train_samples_per_second': 8.121, 'train_steps_per_second': 1.015, 'total_flos': 2105638502400000.0, 'train_loss': 1.726384521484375, 'epoch': 1.0})

In [21]:
trainer.evaluate()

{'eval_loss': 1.3485186100006104,
 'eval_runtime': 72.9913,
 'eval_samples_per_second': 27.401,
 'eval_steps_per_second': 3.425,
 'epoch': 1.0}

# Step 8: Inference

Use the fine-tuned model for inference. Create a pipeline for masked language modeling and test it with a sample sentence.

In [22]:
model.save_pretrained("ourModel")
tokenizer.save_pretrained("ourModel")

('ourModel/tokenizer_config.json',
 'ourModel/special_tokens_map.json',
 'ourModel/vocab.txt',
 'ourModel/added_tokens.json',
 'ourModel/tokenizer.json')

In [28]:
from transformers import pipeline

MLM = pipeline("fill-mask", model="/content/ourModel")
MLM("Hello I'm a [MASK] model.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.19057323038578033,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello i'm a role model."},
 {'score': 0.0713905394077301,
  'token': 1039,
  'token_str': 'c',
  'sequence': "hello i'm a c model."},
 {'score': 0.06968748569488525,
  'token': 1038,
  'token_str': 'b',
  'sequence': "hello i'm a b model."},
 {'score': 0.06254801899194717,
  'token': 2047,
  'token_str': 'new',
  'sequence': "hello i'm a new model."},
 {'score': 0.02190079353749752,
  'token': 1055,
  'token_str': 's',
  'sequence': "hello i'm a s model."}]

In [29]:
MLM('A food manufacturer aim to [MASK] consumer trust')

[{'score': 0.15961501002311707,
  'token': 1041,
  'token_str': 'e',
  'sequence': 'a food manufacturer aim to e consumer trust'},
 {'score': 0.0812612771987915,
  'token': 1037,
  'token_str': 'a',
  'sequence': 'a food manufacturer aim to a consumer trust'},
 {'score': 0.07072862982749939,
  'token': 1051,
  'token_str': 'o',
  'sequence': 'a food manufacturer aim to o consumer trust'},
 {'score': 0.054717693477869034,
  'token': 5326,
  'token_str': 'promote',
  'sequence': 'a food manufacturer aim to promote consumer trust'},
 {'score': 0.035475920885801315,
  'token': 1055,
  'token_str': 's',
  'sequence': 'a food manufacturer aim to s consumer trust'}]